In [10]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os

profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList', 2) 
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.helperApps.neverAsk.openFile', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
profile.set_preference('browser.helperApps.alwaysAsk.force', False)
profile.set_preference('browser.download.manager.alertOnEXEOpen', False)
profile.set_preference('browser.download.manager.focusWhenStarting', False)
profile.set_preference('browser.download.manager.useWindow', False)
profile.set_preference('browser.download.manager.showAlertOnComplete', False)
profile.set_preference('browser.download.manager.closeWhenDone', False)

In [3]:
def main(target_url):
    browser=webdriver.Firefox(firefox_profile=profile)
    browser.get(target_url)
    element=browser.find_element(By.ID,"ViewerControl1_TreeViewFiles")
#    element.send_keys("typing")

    link_txt = []
    for e in element.find_elements_by_class_name('TreeNode'):
#         print(e.text)
        if e.text.endswith('.tif') or e.text.endswith('.TIF'):
#             print('    .tif')
            if e.text not in link_txt:
                link_txt.append(e.text)
#                 print('    added to list')
            else:
                pass
#                 print('    already on list')
        else:
            pass
#             print('    not .tif')
    
    for a in link_txt:
#         print(a)
        lnk = browser.find_element_by_xpath("//*[text()='{}']".format(a))
#         print(lnk)
        lnk.click()
        time.sleep(5)
        dl_button = browser.find_element(By.ID, "ViewerControl1_HyperLinkDownload")
        dl_button.click()
        

    browser.close()

In [4]:
hist_photos_pt= pd.read_excel('Historical photos.xlsx',header=1)['Keywords.2']

In [5]:
hist_photos_pt= hist_photos_pt.dropna().to_list()

In [6]:
hist_photos_pt_cleaned= []
for i in hist_photos_pt:
    i = i.replace('\xa0','')
    i= i.replace(' ','+')
    hist_photos_pt_cleaned.append(i)

In [7]:
hist_photos_pt_cleaned=hist_photos_pt_cleaned[1:-3]

In [8]:
hist_photos_pt_cleaned

['Implantação+da+República',
 'Revolução+1910',
 'Barricada',
 'Dia+da+Liberdade',
 '25+de+Abril+1974',
 'Dia+do+Trabalhador',
 '1º+de+Maio+de+1974',
 'Protesto+trabalhista',
 'Revolução+dos+Cravos',
 'protestos+ambientalistas',
 'lutanuclearem+Portugal',
 'movimentoanti-nuclear+deFerrel',
 'protesto+Ferrel',
 'Ferrel,+1978',
 '1º+de+Maio+em+Liberdade',
 'Pela+Vida+Contra+o+Nuclear',
 'Manifestação',
 'protesto+popular']

In [11]:
for search_term in hist_photos_pt_cleaned:
    try:
        print(search_term)
        basic_link= 'https://digitarq.cpf.arquivos.pt'
        result_link= '/results?p0=UnitTitle&o0=1&v0='
        search_term=search_term
        link_final= basic_link +result_link+ search_term

        final_urls= []
        for base_link in range(100):
            page_part_one= '&p='
            page_num=str(base_link)
            page_part_two='&s=CompleteUnitId&sd=False'

            page= link_final+ page_part_one+page_num+page_part_two

            reqs = requests.get(page)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            prefix= 'https://digitarq.cpf.arquivos.pt/'
            txt= 'href="details?.*"\[[^\s]*\s'
            for i in re.findall(txt,str(soup)):
                final_url= prefix + i.split(" id")[0].replace("details","viewer")
                final_url= final_url.replace('href="','')
                final_url=final_url.replace('"','')
                final_urls.append(final_url)

        search_term= search_term.replace("+","_")

        new_dir= os.path.join(os.getcwd(),"images_from_the_past","PORTUGAL",search_term)
        os.mkdir(new_dir)
        profile.set_preference('browser.download.dir', new_dir)

        for link in final_urls:
            try:
                main(link)
            except:
                print('sth happened!')
                time.sleep(2)
                continue
    except:
        continue

Implantação+da+República
Revolução+1910
Barricada
Dia+da+Liberdade
25+de+Abril+1974
Dia+do+Trabalhador
1º+de+Maio+de+1974
Protesto+trabalhista
Revolução+dos+Cravos
protestos+ambientalistas
lutanuclearem+Portugal
movimentoanti-nuclear+deFerrel
protesto+Ferrel
Ferrel,+1978
1º+de+Maio+em+Liberdade
Pela+Vida+Contra+o+Nuclear
Manifestação
protesto+popular


In [ ]:
# df = pd.read_csv("digitarq_pt/fuzzable.txt", sep=" ",header=None)

# df['links']=df[0]

# df.drop(columns=[0],inplace=True)

# links_df= df[df['links'].str.contains('viewer')]

# link_list =links_df['links'].to_list()